# Deploying Solutions 

- For on-premise, all of these are **open-source** options only

# On-Premise

## Option 1: Docker Compose: Backend _ Frontend

### 📁 Folder Structure (Example)

```bash
project-root/
│
├── backend/
│   ├── main.py
│   ├── requirements.txt
│   └── Dockerfile
│
├── frontend/
│   ├── Dockerfile
│   ├── package.json
│   ├── public/
│   └── src/
│
└── docker-compose.yml
```


### 🔧 Step 1: Backend Dockerfile (e.g., FastAPI)
`backend/Dockerfile`:

```bash
FROM python:3.10

WORKDIR /app
COPY . .
RUN pip install -r requirements.txt

CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]
```



### 🔧 Step 2: Frontend Dockerfile (e.g., React / Next.js)
`frontend/Dockerfile`

```bash
FROM node:18 AS builder

WORKDIR /app
COPY . .
RUN npm install
RUN npm run build

# Serve with Nginx
FROM nginx:stable-alpine
COPY --from=builder /app/build /usr/share/nginx/html
EXPOSE 80
CMD ["nginx", "-g", "daemon off;"]
```


### 🔧 Step 3: Docker Compose File
`docker-compose.yaml`

```bash
version: '3.8'

services:
  backend:
    build:
      context: ./backend
    ports:
      - "8000:8000"
    volumes:
      - ./backend:/app
    environment:
      - ENV=production
      - OPENAI_API_KEY=your_key_here

  frontend:
    build:
      context: ./frontend
    ports:
      - "3000:80"  # React default -> Nginx
    depends_on:
      - backend
```


### ▶️ Step 4: Build & Run

```bash
docker-compose up --build
```

- Frontend → accessible at `http://localhost:3000`
- Backend → accessible at `http://localhost:800`

If you're using fetch() or axios in your frontend to talk to the backend:

```javascript
// Example in frontend/src/config.js
export const API_BASE_URL = "http://localhost:8000";
```

## Option 2: Kubernetes Cluster

Using **MicroK8s** (lightweight K8s distro by Canonical):

### 1. Install MicroK8s:

```bash
sudo snap install microk8s --classic
sudo usermod -a -G microk8s $USER
```

### 2. Enable services

```bash
microk8s enable dns dashboard registry ingress
```

### 3. Build and push container to dockerhub

```bash
docker build -t ai-agent .
docker tag ai-agent localhost:32000/ai-agent
docker push localhost:32000/ai-agent
```

### 4. Create Kubernetes depoloyment YAML (`deployment.yaml`)

```bash
apiVersion: apps/v1
kind: Deployment
metadata:
  name: ai-agent
spec:
  replicas: 1
  selector:
    matchLabels:
      app: ai-agent
  template:
    metadata:
      labels:
        app: ai-agent
    spec:
      containers:
      - name: ai-agent
        image: localhost:32000/ai-agent
        ports:
        - containerPort: 8000
---
apiVersion: v1
kind: Service
metadata:
  name: ai-agent-service
spec:
  selector:
    app: ai-agent
  ports:
    - protocol: TCP
      port: 80
      targetPort: 8000
  type: LoadBalancer
```

### 5. Apply deployment:

```bash
microk8s kubectl apply -f deployment.yaml
```

### 6. Access app via `localhost` or node IP

## Option 3: Nginx + Gunicorn / uvicorn + systemd

### 1. Install necessary packages:
```bash
sudo apt update
sudo apt install python3-pip nginx
pip install uvicorn fastapi
```

### 2. Create a systemd service:
```bash
sudo nano /etc/systemd/system/ai-agent.service
```
Example:
```ini
[Unit]
Description=AI Agent FastAPI Service
After=network.target

[Service]
User=ubuntu
WorkingDirectory=/home/ubuntu/ai-agent
ExecStart=/usr/local/bin/uvicorn main:app --host 127.0.0.1 --port 8000
Restart=always

[Install]
WantedBy=multi-user.target
```

### 3. Enable and start
```bash
sudo systemctl daemon-reexec
sudo systemctl enable ai-agent
sudo systemctl start ai-agent
```

### 4. Configure Nginx as reverse proxy (as in Droplet steps).

### 5. (Optional) Add Let’s Encrypt SSL.

# Cloud Deployment

## ☁️ Cloud Deployment on Microsoft Azure

- **Azure App Service** → for your backend (e.g., FastAPI, Django, Flask)
- **Azure Static Web Apps** or **App Service** → for your frontend (e.g., React, Next.js, HTML)
- **Azure Storage** or **Azure ML** → optional for model hosting or file storage

### 🛠️ Prerequisites

1. Azure account with access to Azure Portal
2. Code hosted on GitHub, Azure Repos, or local Git
3. Azure CLI installed (optional but recommended)

### 🔹 A. DEPLOYING THE BACKEND (AI Agent API)

#### 1. 🔸 Prepare the App

For example, FastAPI app with `main.py`, `requirements.txt`, etc.

If you're using Django or Flask, just replace accordingly.
Make sure `requirements.txt` contains:

```txt
fastapi
uvicorn
```

Your app should have an entry point like:
```bash
uvicorn main:app --host 0.0.0.0 --port 8000
```

#### 2. 🔸Create a Web App on Azure

Either on the Azure portal or through CLI:

```bash
az webapp up --name my-ai-agent-api --runtime "PYTHON|3.10" --resource-group MyResourceGroup --location eastus
```

#### 3. Deploy the Code

##### Option A: GitHub Integration (CI/CD)

1. In App Service, go to Deployment Center
2. Connect GitHub → Choose repo and branch
3. Select Python build and auto-deploy

##### Option B: Manual via Azure CLI

```bash
az webapp up --name my-ai-agent-api --sku B1
```

#### 4. 🔸 Set Environment Variables 

In Azure Portal:
- App Service
- Configuration > Application settings
- Add secrets in TOML
- Save and restart

### B. DEPLOYING THE FRONTEND

#### 🔸 Option 1: Azure Static Web Apps

- Best for static frontend apps (React, HTML, etc.)

1. Go to Azure Portal → Search Static Web Apps → Create

2. Fill in:

    - App Name: my-ai-frontend

    - Source: GitHub

    - Build Preset: React / Next.js / Vue / HTML

    - Location of app: e.g. frontend/ or dist/

3. Configure CI/CD in GitHub

4. Auto-deploys on push

✅ Azure provides global CDN, HTTPS, and custom domains.

#### Option 2: Azure App Service for full-stack apps

If your frontend is tightly integrated (e.g., SSR with Django or Next.js), deploy using the same App Service or create a second one.

Use Node.js or static site configuration if not using Python.


### C. ADDITIONAL SERVICES
🧠 Azure Machine Learning

- If your AI Agent uses a trained ML model and you want to host it:

    1. Register your model on Azure ML

    2. Deploy it as a Real-Time Endpoint

    3. Integrate the endpoint into your backend API


### 🔹 D. DOMAIN + SSL

For both backend and frontend:

1. In Azure Portal → Go to your App Service

2. Add Custom Domain

3. Enable HTTPS Only

4. Azure provides free SSL certificates

### 🔹 E. MONITORING & SCALING
- Enable Application Insights for logs, error tracking

- Scale App Service (CPU/RAM) with Scaling Plan (SKU): B1, P1V2, etc.

- Set up autoscaling based on traffic

### Summary
| Component     | Service                  | Type                    |
| ------------- | ------------------------ | ----------------------- |
| Backend (API) | Azure App Service        | Python app              |
| Frontend      | Azure Static Web Apps    | React/HTML/CSS          |
| Storage       | Azure Blob Storage       | Object storage          |
| ML Model      | Azure Machine Learning   | Optional (hosted model) |
| CI/CD         | GitHub Actions via Azure | Auto-deploy             |

## Cloud Deployment on AWS